# Threshold images

K.Garner, 2020

This wf takes an spmT image, anatomically masks it. The remaining is saved.
Designed to be repeated over t's and over masks.

In [1]:
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
import nipype.interfaces.fsl.maths as fsl
from nipype.interfaces import spm as spm
from nipype.algorithms.misc import Gunzip 
import pandas as pd
import os, re, json
# https://nipype.readthedocs.io/en/0.11.0/users/spmmcr.html

matlab_cmd = '/opt/spm12-r7219/run_spm12.sh /opt/matlabmcr-2010a/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

210701-10:47:47,943 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
thrsh = pe.Workflow(name="thrsh") # workflow to run the analysis

## Data grabbing and sinking

In [3]:
dg = pe.Node(DataGrabber(infields=['sub', 'TR'], 
                         outfields=['ts','msks']), name='data-grabber')
dg.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'ts': [['sub', 'TR']],
                           'msks': [['sub', 'TR']]}
dg.inputs.field_template = {'msks': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-%s/TR%s/MASKS/*.nii.gz',
                            'ts': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-%s/TR%s/FLGLM/spmF_000*.nii'}

In [4]:
# # debugging
dg.inputs.sub = '01'
dg.inputs.TR = '700'
res = dg.run()
res.outputs
ncon = 1

210701-10:47:50,380 nipype.workflow INFO:
	 [Node] Setting-up "data-grabber" in "/tmp/tmpekt4xev3/data-grabber".
210701-10:47:50,386 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
210701-10:47:50,392 nipype.workflow INFO:
	 [Node] Finished "data-grabber".


In [5]:
res.outputs


msks = ['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rCN_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rFEF_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rGPe_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rGPi_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rIPS_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rLOC_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rPut_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rSTN_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rVS_trans.nii.gz']
ts = ['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/spmF_0001.nii', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/spmF_0002.nii', '/scratch/qbi/uqkgarn1/STRIWP1/

In [6]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and TR strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath[0])
    l = fname.split(os.sep)
    name = [s for s in l if re.search('sub', s)][0], [t for t in l if re.search('TR([0-9])', t)][0]
    name = '/'.join(name)
    name
    return name

In [7]:
printSubPath(res.outputs.msks)

'sub-01/TR700'

In [8]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/"
substitutions = [('_TR_([0-9]*)ECHO([0-9]*)_sub_([0-9]*)', ''),
                 ('_TR_([0-9]*)_sub_([0-9]*)', ''),
                 ('_thrsh-t([0-9]*)', ''),
                 ('_absthat([0-9]*)', ''),
                 ('_mask-t([0-9])*', ''),
                 ('_multi([0-9])*', '')] 
ds.inputs.regexp_substitutions = substitutions

In [9]:
datinf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                 name='dat-info')
datinf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]
#datinf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700ECHO1', '700ECHO2'])]
#datinf.iterables = [('sub', ['05']), ('TR', ['700'])]
datinf.iterables 

[('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]

## Gzip interface

get .nii -> .nii.gz

In [10]:
from nipype.interfaces.base import (
    TraitedSpec,
    CommandLineInputSpec,
    CommandLine,
    File
)
import os

class GZipInputSpec(CommandLineInputSpec):
    in_file = File(desc="File", exists=True, mandatory=True, argstr="%s")

class GZipOutputSpec(TraitedSpec):
    out_file = File(desc = "Zip file", exists = True)

class GZipTask(CommandLine):
    input_spec = GZipInputSpec
    output_spec = GZipOutputSpec
    _cmd = 'gzip'

    def _list_outputs(self):
            outputs = self.output_spec().get()
            outputs['out_file'] = os.path.abspath(self.inputs.in_file + ".gz")
            return outputs
        
gzip = pe.MapNode(GZipTask(), iterfield='in_file', name='gzip')

## functions for list cats

In [11]:
def tcat(fullFileList, nMasks=9):
    #return [fullFileList]*nMasks
    return fullFileList*nMasks # use this line when n fullFileList > 1

In [12]:
tcat(res.outputs.ts)

['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/spmF_0001.nii',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/spmF_0002.nii',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/spmF_0003.nii',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/spmF_0001.nii',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/spmF_0002.nii',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/spmF_0003.nii',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/spmF_0001.nii',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/spmF_0002.nii',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/spmF_0003.nii',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/spmF_0001.nii',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/spmF

In [13]:
def maskcat(fullFileList, nCon=3):
    import numpy as np
    return np.ndarray.tolist(np.repeat(fullFileList, nCon))

In [14]:
maskcat(res.outputs.msks)

['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rCN_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rCN_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rCN_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rFEF_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rFEF_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rFEF_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rGPe_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rGPe_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rGPe_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rGPi_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rGPi_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/gl

## define output names for t x msk

In [15]:
#tnms = ['abs_spmT_0001', 'abs_spmT_0002', 'abs_spmT_0005', 'abs_spmT_0007', 'abs_spmT_0013', 'abs_spmT_0014', 'abs_spmT_0015']
#tnms = ['tSNR_beta_0001', 'tSNR_beta_0002', 'tSNR_beta_0003', 'tSNR_beta_0004', 'tSNR_beta_0013', 'tSNR_beta_0014']
#tnms = ['spmT_0004', 'spmT_0005', 'spmT_0006']
tnms = ['spmF_0001', 'spmF_0002', 'spmF_0003']
#tnms = ['spmF_0001']
msks = ['rCN', 'rFEF', 'rGPe', 'rGPi', 'rIPS', 'rLOC', 'rPut', 'rSTN', 'rVS']
touts = ['th'+i+'_'+n+'.nii.gz' for i in msks for n in tnms]

In [16]:
touts

['thrCN_spmF_0001.nii.gz',
 'thrCN_spmF_0002.nii.gz',
 'thrCN_spmF_0003.nii.gz',
 'thrFEF_spmF_0001.nii.gz',
 'thrFEF_spmF_0002.nii.gz',
 'thrFEF_spmF_0003.nii.gz',
 'thrGPe_spmF_0001.nii.gz',
 'thrGPe_spmF_0002.nii.gz',
 'thrGPe_spmF_0003.nii.gz',
 'thrGPi_spmF_0001.nii.gz',
 'thrGPi_spmF_0002.nii.gz',
 'thrGPi_spmF_0003.nii.gz',
 'thrIPS_spmF_0001.nii.gz',
 'thrIPS_spmF_0002.nii.gz',
 'thrIPS_spmF_0003.nii.gz',
 'thrLOC_spmF_0001.nii.gz',
 'thrLOC_spmF_0002.nii.gz',
 'thrLOC_spmF_0003.nii.gz',
 'thrPut_spmF_0001.nii.gz',
 'thrPut_spmF_0002.nii.gz',
 'thrPut_spmF_0003.nii.gz',
 'thrSTN_spmF_0001.nii.gz',
 'thrSTN_spmF_0002.nii.gz',
 'thrSTN_spmF_0003.nii.gz',
 'thrVS_spmF_0001.nii.gz',
 'thrVS_spmF_0002.nii.gz',
 'thrVS_spmF_0003.nii.gz']

## FSL functionality to apply anatomical mask

In [17]:
# maskt = pe.MapNode(fsl.ApplyMask(), iterfield = ['in_file', 'mask_file', 'out_file'], name='mask-t')
# maskt.inputs.out_file = touts
# maskt.inputs.nan2zeros = True
maskt = pe.MapNode(fsl.BinaryMaths(), iterfield = ['in_file', 'operand_file', 'out_file'], name='mask-t')
maskt.inputs.operation = 'mul'
maskt.inputs.nan2zeros = True
maskt.inputs.out_file = touts

## Connect workflow

In [18]:
thrsh.connect([(datinf, dg, [('sub', 'sub')]),
               (datinf, dg, [('TR', 'TR')]),
               (dg, ds, [(('msks', printSubPath), 'container')]),
               (dg, maskt, [(('ts', tcat), 'in_file')]), 
               (dg, maskt, [(('msks', maskcat), 'operand_file')]),
               (maskt, ds, [('out_file', 'THRSH')])
])
res = thrsh.run()

210701-10:47:50,501 nipype.workflow INFO:
	 Workflow thrsh settings: ['check', 'execution', 'logging', 'monitoring']
210701-10:47:50,568 nipype.workflow INFO:
	 Running serially.
210701-10:47:50,569 nipype.workflow INFO:
	 [Node] Setting-up "thrsh.data-grabber" in "/tmp/tmpekt4xev3/data-grabber".
210701-10:47:50,575 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
210701-10:47:50,580 nipype.workflow INFO:
	 [Node] Finished "thrsh.data-grabber".
210701-10:47:50,581 nipype.workflow INFO:
	 [Node] Setting-up "thrsh.mask-t" in "/tmp/tmpbmkgeuzv/thrsh/_TR_1920_sub_05/mask-t".
210701-10:47:50,595 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t0" in "/tmp/tmpbmkgeuzv/thrsh/_TR_1920_sub_05/mask-t/mapflow/_mask-t0".
210701-10:47:50,600 nipype.workflow INFO:
	 [Node] Running "_mask-t0" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM/s

210701-10:47:53,530 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t13" in "/tmp/tmpbmkgeuzv/thrsh/_TR_1920_sub_05/mask-t/mapflow/_mask-t13".
210701-10:47:53,535 nipype.workflow INFO:
	 [Node] Running "_mask-t13" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM/spmF_0002.nii -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/MASKS/rIPS_trans.nii.gz thrIPS_spmF_0002.nii.gz
210701-10:47:53,736 nipype.workflow INFO:
	 [Node] Finished "_mask-t13".
210701-10:47:53,739 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t14" in "/tmp/tmpbmkgeuzv/thrsh/_TR_1920_sub_05/mask-t/mapflow/_mask-t14".
210701-10:47:53,744 nipype.workflow INFO:
	 [Node] Running "_mask-t14" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM/spmF_0003.nii -nan -mul /scratc

210701-10:47:56,541 nipype.workflow INFO:
	 [Node] Finished "thrsh.mask-t".
210701-10:47:56,542 nipype.workflow INFO:
	 [Node] Setting-up "thrsh.sink-stuff" in "/tmp/tmpjlzy39ro/thrsh/_TR_1920_sub_05/sink-stuff".
210701-10:47:56,553 nipype.workflow INFO:
	 [Node] Running "sink-stuff" ("nipype.interfaces.io.DataSink")
210701-10:47:56,555 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/THRSH/_TR_1920_sub_05/_mask-t0/thrCN_spmF_0001.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/THRSH///thrCN_spmF_0001.nii.gz
210701-10:47:56,557 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/THRSH/_TR_1920_sub_05/_mask-t1/thrCN_spmF_0002.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/THRSH///thrCN_spmF_0002.nii.gz
210701-10:47:56,559 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/THRSH/_TR_

210701-10:47:56,611 nipype.workflow INFO:
	 [Node] Setting-up "thrsh.mask-t" in "/tmp/tmpn1bbq4kg/thrsh/_TR_1510_sub_05/mask-t".
210701-10:47:56,625 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t0" in "/tmp/tmpn1bbq4kg/thrsh/_TR_1510_sub_05/mask-t/mapflow/_mask-t0".
210701-10:47:56,630 nipype.workflow INFO:
	 [Node] Running "_mask-t0" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1510/FLGLM/spmF_0001.nii -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1510/MASKS/rCN_trans.nii.gz thrCN_spmF_0001.nii.gz
210701-10:47:56,874 nipype.workflow INFO:
	 [Node] Finished "_mask-t0".
210701-10:47:56,877 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t1" in "/tmp/tmpn1bbq4kg/thrsh/_TR_1510_sub_05/mask-t/mapflow/_mask-t1".
210701-10:47:56,882 nipype.workflow INFO:
	 [Node] Running "_mask-t1" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with comm

210701-10:47:59,884 nipype.workflow INFO:
	 [Node] Finished "_mask-t13".
210701-10:47:59,887 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t14" in "/tmp/tmpn1bbq4kg/thrsh/_TR_1510_sub_05/mask-t/mapflow/_mask-t14".
210701-10:47:59,892 nipype.workflow INFO:
	 [Node] Running "_mask-t14" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1510/FLGLM/spmF_0003.nii -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1510/MASKS/rIPS_trans.nii.gz thrIPS_spmF_0003.nii.gz
210701-10:48:00,126 nipype.workflow INFO:
	 [Node] Finished "_mask-t14".
210701-10:48:00,128 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t15" in "/tmp/tmpn1bbq4kg/thrsh/_TR_1510_sub_05/mask-t/mapflow/_mask-t15".
210701-10:48:00,134 nipype.workflow INFO:
	 [Node] Running "_mask-t15" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/der

...

210701-10:49:08,431 nipype.workflow INFO:
	 [Node] Finished "_mask-t16".
210701-10:49:08,434 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t17" in "/tmp/tmpl5kbsyku/thrsh/_TR_1510_sub_01/mask-t/mapflow/_mask-t17".
210701-10:49:08,439 nipype.workflow INFO:
	 [Node] Running "_mask-t17" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/FLGLM/spmF_0003.nii -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/MASKS/rLOC_trans.nii.gz thrLOC_spmF_0003.nii.gz
210701-10:49:08,639 nipype.workflow INFO:
	 [Node] Finished "_mask-t17".
210701-10:49:08,642 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t18" in "/tmp/tmpl5kbsyku/thrsh/_TR_1510_sub_01/mask-t/mapflow/_mask-t18".
210701-10:49:08,646 nipype.workflow INFO:
	 [Node] Running "_mask-t18" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/der

210701-10:49:10,870 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/THRSH/_TR_1510_sub_01/_mask-t7/thrGPe_spmF_0002.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/THRSH///thrGPe_spmF_0002.nii.gz
210701-10:49:10,872 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/THRSH/_TR_1510_sub_01/_mask-t8/thrGPe_spmF_0003.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/THRSH///thrGPe_spmF_0003.nii.gz
210701-10:49:10,873 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/THRSH/_TR_1510_sub_01/_mask-t9/thrGPi_spmF_0001.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/THRSH///thrGPi_spmF_0001.nii.gz
210701-10:49:10,875 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/THRSH/_TR_1510_sub_01/_mask-t10/thrGPi_spmF_000

210701-10:49:11,777 nipype.workflow INFO:
	 [Node] Finished "_mask-t3".
210701-10:49:11,779 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t4" in "/tmp/tmp6t99ty4q/thrsh/_TR_700_sub_01/mask-t/mapflow/_mask-t4".
210701-10:49:11,784 nipype.workflow INFO:
	 [Node] Running "_mask-t4" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/spmF_0002.nii -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rFEF_trans.nii.gz thrFEF_spmF_0002.nii.gz
210701-10:49:11,952 nipype.workflow INFO:
	 [Node] Finished "_mask-t4".
210701-10:49:11,954 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t5" in "/tmp/tmp6t99ty4q/thrsh/_TR_700_sub_01/mask-t/mapflow/_mask-t5".
210701-10:49:11,959 nipype.workflow INFO:
	 [Node] Running "_mask-t5" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm

210701-10:49:14,233 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t18" in "/tmp/tmp6t99ty4q/thrsh/_TR_700_sub_01/mask-t/mapflow/_mask-t18".
210701-10:49:14,238 nipype.workflow INFO:
	 [Node] Running "_mask-t18" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/spmF_0001.nii -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rPut_trans.nii.gz thrPut_spmF_0001.nii.gz
210701-10:49:14,397 nipype.workflow INFO:
	 [Node] Finished "_mask-t18".
210701-10:49:14,400 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t19" in "/tmp/tmp6t99ty4q/thrsh/_TR_700_sub_01/mask-t/mapflow/_mask-t19".
210701-10:49:14,405 nipype.workflow INFO:
	 [Node] Running "_mask-t19" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/spmF_0002.nii -nan -mul /scratch/qbi

210701-10:49:15,764 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/THRSH/_TR_700_sub_01/_mask-t10/thrGPi_spmF_0002.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/THRSH///thrGPi_spmF_0002.nii.gz
210701-10:49:15,766 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/THRSH/_TR_700_sub_01/_mask-t11/thrGPi_spmF_0003.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/THRSH///thrGPi_spmF_0003.nii.gz
210701-10:49:15,767 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/THRSH/_TR_700_sub_01/_mask-t12/thrIPS_spmF_0001.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/THRSH///thrIPS_spmF_0001.nii.gz
210701-10:49:15,769 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/THRSH/_TR_700_sub_01/_mask-t13/thrIPS_spmF_0002.nii.gz